# Default notebook

This default notebook is executed using Databricks Workflows as defined in resources/delta_live_store_job.yml.

In [ ]:
%pip install -qr ../requirements-dev.txt 
%pip install -q --upgrade databricks-sdk
dbutils.library.restartPython()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Callable, List

from dbrx.dls.store import DeltaLiveStore, DeltaLiveEntity, DeltaLiveEntityList
from dbrx.dls.filters import by_is_enabled, by_is_latest, Predicate
from dbrx.dls.loader import DeltaLiveStoreLoader, FileServiceLoader, entity_from_directory_entry, medallion_from_directory_entry

In [7]:
dbutils.widgets.text("host", "")
dbutils.widgets.text("token", "")
dbutils.widgets.text("volume", "")
dbutils.widgets.text("host", "")
dbutils.widgets.text("deltaLiveStoreTable", "")
dbutils.widgets.text("deltaLiveStoreEntityGroup", "")

In [ ]:

host: str = dbutils.widgets.get("host")
token: str = dbutils.widgets.get("token")
volume: str = dbutils.widgets.get("volume")
delta_live_store_table: str = dbutils.widgets.get("deltaLiveStoreTable")
delta_live_store_entity_group: str = dbutils.widgets.get("deltaLiveStoreEntityGroup")

if not host:
    host = spark.conf.get("spark.databricks.workspaceUrl", None)
if not token:
    token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

host = host or None
token = token or None
volume = volume or None
delta_live_store_table = delta_live_store_table or None
delta_live_store_entity_group = delta_live_store_entity_group or None

print(f"host: {host}")
print(f"token: {token}")
print(f"volume: {volume}")
print(f"delta_live_store_table: {delta_live_store_table}")
print(f"delta_live_store_entity_group: {delta_live_store_entity_group}")

if host is None:
    raise ValueError("host is not set")
if token is None:
    raise ValueError("token is not set")
if delta_live_store_table is None:
    raise ValueError("deltaLiveStoreTable is not set")
if volume is None:
    raise ValueError("volume is not set")

In [ ]:
store: DeltaLiveStore = DeltaLiveStore(delta_live_store_table)
store.initialize()

In [ ]:
bronze_factory: Callable[["DirectoryEntry"], List[DeltaLiveEntity]] = entity_from_directory_entry()
medallion_factory: Callable[["DirectoryEntry"], List[DeltaLiveEntity]] = medallion_from_directory_entry()

loader: DeltaLiveStoreLoader = FileServiceLoader(volume, host=host, token=token, factory=medallion_factory)

loader.load(store)

In [ ]:
predicate: Predicate = by_is_latest() & by_is_enabled()
entities: DeltaLiveEntityList = store.find_entities(predicate)
display(entities.to_df())